In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

def get_bbox(file_path):
    bbox_dict = {}
    with open(file_path) as json_file:
        json_data = json.load(json_file)
        bbox_dict["name"] = json_data["images"][0]["file_name"]
        bbox_dict["annotations"] = []
        annotations = json_data["annotations"]
        for annotation in annotations:
            bbox = {}
            bbox["id"] = annotation["id"]
            bbox["coord"] = annotation["bbox"]
            bbox["label"] = annotation["text"]
            if bbox["label"] == "xxx":
                continue
            bbox_dict["annotations"].append(bbox)

    return bbox_dict

In [37]:
import os
import cv2

json_dir = "/content/drive/MyDrive/project3/deep-text-recognition-benchmark/2020-02-030.야외실제촬영한글_sample/라벨링데이터/"
img_dir = "/content/drive/MyDrive/project3/deep-text-recognition-benchmark/2020-02-030.야외실제촬영한글_sample/원천데이터/"
result_dir_train = "/content/drive/MyDrive/project3/deep-text-recognition-benchmark/data/train/kordata"
result_dir_val = "/content/drive/MyDrive/project3/deep-text-recognition-benchmark/data/val/kordata"
img_list = []
for root, dirs, files in os.walk(json_dir):
    for file in files:
        img_list.append(get_bbox("{}/{}".format(root,file)))

train_cnt = round(len(img_list) * 0.9)
img_train = img_list[:train_cnt]
img_val = img_list[train_cnt:]

print("img_train : ",len(img_train))
print("img_val : ",len(img_val))

gt_train = []
gt_val = []

# make train dataset
for img in img_train:
    img_name = img["name"]
    original_img = cv2.imread(img_dir+img_name)
    for annotation in img["annotations"]:
        cropped_img_name = "{}_{}.{}".format(img_name.split('.')[0], str(annotation["id"]), "png")
        x,y,w,h = annotation["coord"]
        cropped_img = original_img[y: y + h, x: x + w].copy()
        cv2.imwrite("{}/images/{}".format(result_dir_train,cropped_img_name),cropped_img)
        gt_train.append("{}/images/{}	{}".format(result_dir_train,cropped_img_name,annotation["label"]))

gt_train_path = result_dir_train + "/gt.txt"
with open(gt_train_path, 'w+') as lf:
    lf.write('\n'.join(gt_train))

# make validation dataset
for img in img_val:
    img_name = img["name"]
    original_img = cv2.imread(img_dir+img_name)
    for annotation in img["annotations"]:
        cropped_img_name = "{}_{}.{}".format(img_name.split('.')[0], str(annotation["id"]), "png")
        x,y,w,h = annotation["coord"]
        cropped_img = original_img[y: y + h, x: x + w].copy()
        cv2.imwrite("{}/images/{}".format(result_dir_val,cropped_img_name),cropped_img)
        gt_val.append("{}/images/{}	{}".format(result_dir_val,cropped_img_name,annotation["label"]))

gt_val_path = result_dir_val + "/gt.txt"
with open(gt_val_path, 'w+') as lf:
    lf.write('\n'.join(gt_val))

img_train :  1026
img_val :  114


# 학습용 데이터 포맷 lmdb 형태로 변환

In [38]:
%cd /content/drive/MyDrive/project3/deep-text-recognition-benchmark

/content/drive/MyDrive/project3/deep-text-recognition-benchmark


In [39]:
!pip install fire

In [ ]:
!python create_lmdb_dataset.py \
--gtFile "data/train/kordata/gt.txt" \
--inputPath "data/trai/kordatan" \
--outputPath "data/lmdb/train"

!python create_lmdb_dataset.py \
--gtFile "data/val/kordata/gt.txt" \
--inputPath "data/val/kordata" \
--outputPath "data/lmdb/val"

error occured 120
Written 1000 / 1458
